In [1]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,roc_auc_score

from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('data/airline_delay_train_new.csv',parse_dates =['FlightDate'])

In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 324836 entries, 236722 to 121958
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   FlightDate     324836 non-null  datetime64[ns]
 1   DepTime        324836 non-null  object        
 2   UniqueCarrier  324836 non-null  object        
 3   Origin         324836 non-null  object        
 4   Dest           324836 non-null  object        
 5   Distance       324836 non-null  int64         
 6   Day_of_Week    324836 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 19.8+ MB


#Process pipeline 
- Separate target from features 
- Pipeline 
- CV split 
- Scores 

In [5]:
y = df['dep_delayed_15min']
X = df.drop(['dep_delayed_15min'], axis=1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split( 
                        X,y,test_size = 0.20, random_state = 42) 

In [8]:
categorical_features = X.select_dtypes(include=['object']).columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns

In [9]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [159]:
from sklearn.ensemble import RandomForestClassifier
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(random_state= 42))])

In [163]:
#from sklearn.model_selection import cross_val_score
#scores = cross_val_score(baseline_pipeline,X, y,cv=3, scoring ='roc_auc')
# the parameters frid expects the same name as the pipeline "classifier"

param_grid = {
    'classifier__max_depth': [80, 100],
    'classifier__max_features': [2, 3],
    'classifier__min_samples_leaf': [3, 4, 5],
    'classifier__min_samples_split': [8, 10, 12],
    'classifier__n_estimators': [100, 200, 300]
}

In [ ]:
def model_fit():
    grid_search = GridSearchCV(estimator = pipe, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
    grid_search.fit(X_train, y_train)
    print ("the best model params: ", grid_search.best_params_)
    best_grid = grid_search.best_estimator_
    return best_grid

In [164]:
grid_search = GridSearchCV(estimator = pipe, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [165]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         SimpleImputer(strategy='constant'),
                                                                         Index(['Distance'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         Index(

In [166]:
grid_search.best_params_

{'classifier__max_depth': 80,
 'classifier__max_features': 2,
 'classifier__min_samples_leaf': 3,
 'classifier__min_samples_split': 8,
 'classifier__n_estimators': 100}

In [167]:
best_grid = grid_search.best_estimator_

In [ ]:
def model_pedict():
    testscore = best_grid.predict_proba(X_test)
    return log_loss(y_test, testscore)

In [169]:
testscore = best_grid.predict_proba(X_test)

In [172]:
log_loss(y_test, testscore)

0.4828014996428915

In [10]:

model = xgb.XGBClassifier()
pipe_xgb = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)])

In [11]:
param_grid_xgboost = {
    'model__n_estimators': [500,700]
}

In [14]:
grid_xgb = GridSearchCV(pipe_xgb, param_grid_xgboost, cv=3, n_jobs=-1, scoring='roc_auc')

In [15]:
grid_xgb.fit(X_train, y_train)

/opt/anaconda3/envs/fsds/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:32:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         SimpleImputer(strategy='constant'),
                                                                         Index(['Distance'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         Index(

In [16]:
best_xgb = grid_xgb.best_estimator_

In [17]:
best_xgb_score = best_xgb.predict_proba(X_test)

In [18]:
log_loss(y_test, best_xgb_score)

0.4529898514021672

In [19]:
test = pd.read_csv('data/airline_delay_test_new.csv')

In [20]:
ext_test = test.drop('dep_delayed_15min', axis=1)

In [21]:
best_xgb_score_test = best_xgb.predict_proba(ext_test)

In [22]:
log_loss(test['dep_delayed_15min'], best_xgb_score_test)

0.4513241335009996

## Python Serialization for persistence 

In [24]:
# import joblib
from joblib import dump

# dump the pipeline model
dump(grid_xgb.best_estimator_, filename="xgboost.joblib")

['xgboost.joblib']

In [26]:
X_train.columns

Index(['FlightDate', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance',
       'Day_of_Week'],
      dtype='object')